In [1]:
#加载数据 
import sys
import os
h_r_folder_name = 'image-dataset/h_r'
l_r_folder_name = 'image-dataset/l_r'
h_r_files_list = os.listdir(h_r_folder_name)
l_r_files_list = os.listdir(l_r_folder_name)
h_r_files_list = [h_r_folder_name+'/'+ x for x in h_r_files_list]
l_r_files_list = [l_r_folder_name+'/'+ x for x in l_r_files_list]

In [2]:
# 展示数据
import numpy as np
import cv2
img_hr = cv2.imread(h_r_files_list[3])
img_lr = cv2.imread(l_r_files_list[3])
cv2.imshow('image',img_lr)
cv2.imshow('image2',img_hr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
# Tcbcr和RGB色彩转换
img_lr = cv2.cvtColor(img_lr, cv2.COLOR_BGR2YCR_CB)
img_hr = cv2.cvtColor(img_hr, cv2.COLOR_BGR2YCR_CB)
cv2.imshow('image',img_lr)
cv2.imshow('image2',img_hr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
# 缩放低分辨率图片用bicubic
# 转换到Ycbcr格式
# 为了做到这些我们需要子样品对儿

no_of_files = len(h_r_files_list)

## 为了subsampling我们设置大小为80px*80px
subimg_length = 32
stride_length = 32
stride_height = 32
subimg_height = 32

sample_img = cv2.imread(h_r_files_list[0])
subimg_for_length = int(np.shape(sample_img)[0])/(subimg_length+stride_length)
subimg_for_height = int(np.shape(sample_img)[1])/(subimg_height+stride_height)

# Final arrays 
X = np.ndarray(shape = (no_of_files*subimg_for_length*subimg_for_height,subimg_length,subimg_height,3))
Y = np.ndarray(shape = (no_of_files*subimg_for_length*subimg_for_height,subimg_length,subimg_height,3)) 

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
for i in range(no_of_files):
    # 缩放图片
    img_lr = cv2.imread(l_r_files_list[i])
    img_lr_zoomed = cv2.resize(img_lr, None,fx=2,fy=2, interpolation = cv2.INTER_CUBIC)
    img_hr = cv2.imread(h_r_files_list[i])
    # 转换为YCbCr
    img_lr_zoomed = cv2.cvtColor(img_lr_zoomed, cv2.COLOR_BGR2YCR_CB)
    img_hr = cv2.cvtColor(img_hr, cv2.COLOR_BGR2YCR_CB)
    #img_lr_zoomed.resize(3,480,320)
    #img_hr.resize(3,480,320)
    # 制作subsamples
    for j in range(subimg_for_length):
        for k in range(subimg_for_height):
            img_current_lr = img_lr_zoomed[j*(stride_length+ subimg_length):j*(stride_length+ subimg_length)+subimg_length,
                                           k*(stride_height+ subimg_height):k*(stride_height+ subimg_height)+subimg_height,:]
            img_current_hr = img_hr[j*(stride_length+ subimg_length):j*(stride_length+ subimg_length)+subimg_length,
                                    k*(stride_height+ subimg_height):k*(stride_height+ subimg_height)+subimg_height,:]
            X[i*subimg_for_height*subimg_for_length+j*subimg_for_height +k] = img_current_lr
            Y[i*subimg_for_height*subimg_for_length+j*subimg_for_height +k] = img_current_hr

In [ ]:
### 有了子图片 现在做3层卷积 filter大小5,1,3
import theano
import theano.tensor as T
import lasagne

#from layer import ConvLayer
#from tools.image_processing import preprocess

In [ ]:
# 准备Theano variables西亚诺变量给input和target
input_var = T.tensor4('inputs')
target_var = T.tensor4('targets')

In [ ]:
network = lasagne.layers.InputLayer(shape=(None, 3,None, None), input_var=input_var)
network = lasagne.layers.Conv2DLayer(
        network, num_filters=32,pad=2, filter_size=(5, 5),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
network = lasagne.layers.Conv2DLayer(
        network, num_filters=32, filter_size=(1, 1),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())
network = lasagne.layers.Conv2DLayer(
        network,pad=1, num_filters=3, filter_size=(3, 3),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=lasagne.init.GlorotUniform())

In [ ]:
#创建一个损失表达for training, i.e., a scalar objective we want
# to minimize (for our multi-class problem, it is the cross-entropy loss):
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.squared_error(prediction, target_var)
loss = loss.mean()

In [ ]:
# 创建一个损失表达for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01,momentum=0.9)

In [ ]:
# 创建一个损失表达for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network,deterministic=True)
test_loss = lasagne.objectives.squared_error(test_prediction,target_var)
test_loss = test_loss.mean()

In [ ]:
# 编译一个函数表达一个训练步骤on a mini-batch通过升级dictionary并返回相应的training损失
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# 编译第二个函数用来计算 验证损失&精度：
val_fn = theano.function([input_var, target_var], [test_loss])
# for true test set
predict_fn = theano.function([input_var],[test_prediction])


In [ ]:
## 改变数据shape
no_of_imgs = int(np.shape(X)[0])
#X.shape(no_of_imgs,3,subimg_length,subimg_height)
X = np.reshape(X,(no_of_imgs,3,subimg_length,subimg_height))
Y = np.reshape(Y,(no_of_imgs,3,subimg_length,subimg_height))
np.shape(Y)

In [ ]:
import time

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
        # 每个epoch 都要full pass训练数据
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X, Y, 100, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        #val_err = 0
        #val_acc = 0
        #val_batches = 0
        #for batch in iterate_minibatches(X, Y, 100, shuffle=False):
        #    inputs, targets = batch
        #    err, acc = val_fn(inputs, targets)
        #    val_err += err
        #    val_acc += acc
        #    val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        #print("  validation accuracy:\t\t{:.2f} %".format(
        #    val_acc / val_batches * 100))


In [ ]:
# 已经训练过CNN
# 和bicubic比较 

img_lr = cv2.imread(l_r_files_list[1],1)
img_hr_bicubic = cv2.resize(img_lr, None,fx=2,fy=2, interpolation = cv2.INTER_CUBIC)
cv2.imshow('image2',img_hr_bicubic)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img_lr = cv2.imread(l_r_files_list[1],1)
img_hr_bicubic = cv2.resize(img_lr, None,fx=2,fy=2, interpolation = cv2.INTER_CUBIC)
img_length = int(np.shape(img_hr_bicubic)[0])
img_height = int(np.shape(img_hr_bicubic)[1])
img_hr_bicubic_ycbcr = cv2.cvtColor(img_hr_bicubic, cv2.COLOR_BGR2YCR_CB)
#img_hr_srcnn = img_hr_bicubic_ycbcr
img_hr_srcnn = predict_fn(img_hr_bicubic_ycbcr)

In [ ]:
img_hr_srcnn_correct_dim = np.reshape(img_hr_srcnn,(img_length,img_height,3))
img_hr_srcnn_correct_dim_BGR = cv2.cvtColor(img_hr_srcnn_correct_dim, cv2.COLOR_YCR_CB2BGR)
cv2.imshow('image2',img_hr_srcnn_correct_dim_BGR)
cv2.waitKey(0)
cv2.destroyAllWindows()